In [1]:
from cyschoolhousesuite import *
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import numpy as np

def click_delete(driver):
    driver.find_element_by_xpath('//input[contains(@value, "Delete")]').click()
    EC.alert_is_present
    driver.switch_to.alert.accept()
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/table/tbody/tr/td[2]/div[2]/form/div/span/span[1]/input')))

sch_ref_path = "Z:\\ChiPrivate\\Chicago Data and Evaluation\\SY18\\SY18 Team Placement\\SY18 FINAL Team Placements.xlsx"
sch_ref_df = pd.read_excel(sch_ref_path, sheetname="School Data")

d = {'Reporting Perdiod': ['Quarter 2', 'Quarter 3', 'Quarter 4'], 
     'Qtr': ['Qtr 2', 'Qtr 3', 'Qtr 4'],
     'Q': ['Q2', 'Q3', 'Q4'],
     'Due Date': ['02/01/2018', '04/12/2018', '06/18/2018']}
gd_assign = pd.DataFrame(d)
subjects = ['Math', 'ELA']

In [46]:
assign_df = get_report('00O1a000002rwMN')
assign_df['key'] = assign_df['Assignment: Section: Section Name'] + assign_df['Reporting Period Name']

#assign_df = assign_df[assign_df['Reporting Period Name']='Quarter 1']
#assign_df.drop_duplicates('Assignment: Assignment ID', inplace=True)
#assign_df = assign_df[assign_df.duplicated('key', keep='first')]

assign_counts = assign_df.groupby(by=['key', 'Assignment: Assignment ID']).count()
assign_counts[assign_counts['Entered Grade']==0]

# BE VERY CAREFUL - COULD DELETE GRADES
#for x in assign_df['Assignment: Assignment ID']:
#    driver.get('https://na24.salesforce.com/' + str(x))
#    click_delete(driver)

In [59]:
# Add Students to Gradebook Sections
driver = get_driver()
open_cyschoolhouse18(driver)

wait = 10

for school in sch_ref_df['School'].tolist():
    print("Starting: {}".format(school))
    driver.get("https://c.na24.visual.force.com/apex/core_ChangeSchool")
    driver.find_element_by_xpath("//select[@name='j_id0:j_id30:j_id32']/option[text()=\"%s\"]" % school).click()
    sleep(3)
    driver.find_element_by_xpath("//input[contains(@class, 'btn') and contains(@value, 'Change')]").click()
    sleep(3)
    for i, r in gd_assign.ix[1:].iterrows():
        for x in subjects:
            driver.get('https://c.na24.visual.force.com/apex/gradebook_addassignments_v2')
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//li[contains(@id, 'addExistingTab')]")))
            driver.find_element_by_xpath("//li[contains(@id, 'addExistingTab')]").click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//select[contains(@name, 'j_id0:mainForm:j_id75:j_id88')]")))
            driver.find_element_by_xpath("//select[@name='j_id0:mainForm:j_id75:j_id88']/option[text()='Reporting Period Course Grade']").click()
            # wait for box to appear/disapper
            fancy_box_wait(waittime=10)
            driver.find_element_by_xpath("//input[@name='j_id0:mainForm:j_id75:j_id84']").send_keys(r['Qtr'])
            driver.find_element_by_xpath("//input[contains(@class, 'btn drk_blue_btn') and contains(@value, 'Search Assignments')]").click()
            fancy_box_wait(waittime=10)
            
            if x=='Math':
                # choose math, enter credentials
                driver.find_element_by_xpath("//td[contains(text(), \"%s\")]" % (r['Qtr'] + " - " + x)).click()
                fancy_box_wait(waittime=10)
                driver.find_element_by_xpath("//input[contains(@name, 'j_id0:mainForm:j_id75:j_id153')]").clear()
                driver.find_element_by_xpath("//input[contains(@name, 'j_id0:mainForm:j_id75:j_id153')]").send_keys(r['Qtr'] + " - " + x + " - Reporting Period Course Grade - Chicago")
                driver.find_element_by_xpath("//input[contains(@name, 'j_id0:mainForm:j_id75:j_id166')]").send_keys(r['Q'] + ' Ma')
                # choose blue
                driver.find_element_by_xpath("//select[@name='j_id0:mainForm:j_id75:j_id211']/option[text()='green']").click()
            
            elif x=='ELA':
                # choose ELA, enter credentials
                driver.find_element_by_xpath("//td[contains(text(), \"%s\")]" % (r['Qtr'] + " - " + x)).click()
                fancy_box_wait(waittime=10)
                driver.find_element_by_xpath("//input[contains(@name, 'j_id0:mainForm:j_id75:j_id153')]").clear()
                driver.find_element_by_xpath("//input[contains(@name, 'j_id0:mainForm:j_id75:j_id153')]").send_keys(r['Qtr'] + " - " + x + " - Reporting Period Course Grade - Chicago")
                driver.find_element_by_xpath("//input[contains(@name, 'j_id0:mainForm:j_id75:j_id166')]").send_keys(r['Q'] + 'ELA')
                # choose blue
                driver.find_element_by_xpath("//select[@name='j_id0:mainForm:j_id75:j_id211']/option[text()='blue']").click()
            
            driver.find_element_by_xpath("//select[@name='j_id0:mainForm:j_id75:j_id232']/option[text()='A-F']").click()
            fancy_box_wait(waittime=10)
            # select all
            driver.find_element_by_xpath("//input[contains(@onclick, \"%s\")]" % "selectAll2(this,'addAssCb');").click()

            for elem in driver.find_elements_by_xpath("//select[contains(@class, 'rPeriod')]"):
                ID=elem.get_attribute('id')
                driver.find_element_by_xpath("//select[@id='{}']/option[text()='{}']".format(ID, r['Reporting Perdiod'])).click()
    
            for elem in driver.find_elements_by_xpath("//input[contains(@class, 'datepicker hasDatepicker')]"):
                elem.clear()
                elem.send_keys(r['Due Date'])
           
            driver.find_element_by_xpath("//input[contains(@class, 'btn black_btn') and contains(@value, 'Submit')]").click()
            
            fancy_box_wait(waittime=10)

Starting: Amos Alonzo Stagg Elementary School of Excellence
Starting: Anna R. Langford Community Academy
Starting: Brian Piccolo Elementary School of Excellence
Starting: Carter G Woodson South Elementary School
Starting: Chalmers School of Excellence
Starting: Charles S Deneen School of Excellence
Starting: Chicago Academy High School
Starting: Collins Academy High School
Starting: Dulles School of Excellence
Starting: Dvorak School of Excellence
Starting: Edmond Burke Elementary School
Starting: Frederick Funston Elementary School
Starting: Fuller School of Excellence
Starting: Gage Park High School
Starting: Gale Community Academy
Starting: Herzl School of Excellence
Starting: John J Pershing Elementary Humanities Magnet
Starting: John Milton Gregory Elementary School
Starting: Johnson School of Excellence
Starting: Jordan Elementary Community School
Starting: Joseph Lovett Elementary School
Starting: McNair School of Excellence
Starting: O'Keeffe School of Excellence
Starting: Spen